In [50]:
import * as F from "fp-ts/function";
import * as A from "fp-ts/Array";
import * as O from "fp-ts/Option";
import { Option } from "fp-ts/Option";

import * as P from "./lib/promise";

import * as E from "fp-ts/Either";
import { Either } from "fp-ts/Either";


Let's write down the types of different `map` functions

In [51]:
type mapArray = <A, B>(f: (a: A) => B) => (arr: Array<A>) => Array<B>;
type mapOption = <A, B>(f: (a: A) => B) => (option: Option<A>) => Option<B>;
type mapPromise = <A, B>(f: (a: A) => B) => (promise: Promise<A>) => Promise<B>;

type mapEither = <A, B>(f: (a: A) => B) => <E>(either: Either<E, A>) => Either<E, B>;

What if we just want to type `map` independently of the data type we're operating on?

We can try to add a type param like so:

In [52]:
//@ts-expect-error
type map = <A, B, T>(f: (a: A) => B) => (t: T<A>) => T<B>;

But this doesn't work, because type script does not allow type arguments to be generic
This is what is called `Higher-Kinded types` (HKT), the ability to have type argument that are themselves generic
   
Let's assume that we could write the above.

We could then define a type:


In [53]:
//@ts-expect-error
type Functor<F> = { map: <A, B>(f: (a: A) => B) => (t: F<A>) => F<B> };

we can then create specific instances of this type for each of the types above:

In [54]:
//@ts-expect-error
const functorArray: Functor<Array> = { map: A.map };
//@ts-expect-error
const functorOption: Functor<Option> = { map: O.map };
//@ts-expect-error
const functorPromise: Functor<Promise> = { map: P.map };

Now, to `map` arrays we could do:

In [55]:
const foo = functorArray.map<number, string>(e => JSON.stringify(e))([1, 2, 3]);

foo

[ '1', '2', '3' ]


* Why is this powerful?

because we could write code that doesn't care about the types it's operating on


In [56]:
//@ts-expect-error
const stringifyAndExclam: <F>(functor: Functor<F>) => <A>(fa: F<A>) => F<string> = functor => functor.map(e => JSON.stringify(e) + "!");

We create a `stringifyAndExclam` function that will work on *any* generic datatype, provided we supply an implementation of the functor interface for said datatype.   
Which means we could do:

In [64]:
const arrayOfStrings = stringifyAndExclam(functorArray)([1, 2]); 

const optionOfString = stringifyAndExclam(functorOption)(O.some(1)); 
const promiseOfStrings = stringifyAndExclam(functorPromise)(Promise.resolve(1)); 

In [58]:
arrayOfStrings

[ '1!', '2!' ]


In [59]:
optionOfString

{ _tag: 'Some', value: '1!' }


In [60]:
await promiseOfStrings

1!


But what about `Either`? It takes 2 type arguments.
In our `functor` type declaration, we have no indication of that, we can see it is slightly different from the `mapEither` type due to the missing `E` type

Again, `TypeScript` doesn't support this, but languages that do support `HKT` typically allow partial application of type arguments.  
That means we could attempt something like:

In [61]:
//@ts-expect-error
type Either1 = Either<E>;
//@ts-expect-error
type Either2 = Either1<A>;

2:23 - Exported type alias 'Either1' has or is using private name 'E'.
4:24 - Exported type alias 'Either2' has or is using private name 'A'.


And this would mean 
```
Either2 = Either<E, A>
```

But TS doesn't allow it. And there are other problems too, as here it implies `E` and `A` are concrete types, but they should be type arguments. There's no way of specifying such constraints.

For the sake of learning, let's go ahead and do our best using `unknown`

In [ ]:
//@ts-expect-error
const functorEither: Functor<Either<unknown>> = { map: E.map };


We specify that `E` is unknown, and this is actually fine, since for the `map` operation, we don't care about those types.

We just want to follow the types defined above. With `unknown`, they would turn out as:

In [ ]:
type mapEither = <A, B>(f: (a: A) => B) => (either: Either<unknown, A>) => Either<unknown, B>;

Of course, ignoring that this doesn't type check, if we were to use these types, we would lose the type information of `E`.

But at least the code works, since we're basically merely ignoring the types and running Javascript underneath

In [ ]:
const rightOfStrings = stringifyAndExclam(functorEither)(E.right(1)); 
const leftOfStrings = stringifyAndExclam(functorEither)(E.left(2)); 


In [62]:
rightOfStrings

{ _tag: 'Right', right: '1!' }


In [63]:
leftOfStrings

{ _tag: 'Left', left: 2 }
